In [1]:
import numpy as np
from squlearn.encoding_circuit import ChebyshevPQC,QCNNEncodingCircuit
from squlearn.observables import SummedPaulis, CustomObservable

pqc = ChebyshevPQC(2, 2, 2)
obs = CustomObservable(2, ["XY", "ZZ", "ZI", "IY"])


np.random.seed(42)
circuit1 = pqc.get_circuit(np.random.rand(pqc.num_features),np.random.rand(pqc.num_parameters))
circuit2 = pqc.get_circuit(np.random.rand(pqc.num_features),np.random.rand(pqc.num_parameters))
observable1 = obs.get_operator(np.random.rand(obs.num_parameters))
observable2 = obs.get_operator(np.random.rand(obs.num_parameters))

In [2]:
circuit1

In [3]:
observable2

SparsePauliOp(['XY', 'ZZ', 'ZI', 'IY'],
              coeffs=[1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j])

In [4]:
from squlearn.util.optree import OpTree,OpTreeList


measured_tree = OpTree.gen_expectation_tree(OpTreeList([circuit1,circuit2]), OpTreeList([observable1,observable2]))
measured_tree = OpTree.evaluate.transform_to_zbasis(measured_tree)

print(measured_tree)

[1.0*[1.0*(1.0*
          ┌─────────────┐┌─────────────┐                 ┌────────────┐»
     q_0: ┤ Ry(0.73199) ├┤ Rx(0.18518) ├───────■─────────┤ Rx(1.0281) ├»
          ├─────────────┤├─────────────┤┌──────┴───────┐┌┴────────────┤»
     q_1: ┤ Ry(0.59866) ├┤ Rx(0.04918) ├┤ Rz(0.058084) ├┤ Rx(0.18951) ├»
          └─────────────┘└─────────────┘└──────────────┘└─────────────┘»
__c_XY: 2/═════════════════════════════════════════════════════════════»
                                                                       »
«                         ┌──────────────┐┌─────┐┌───┐┌─┐
«     q_0: ───────■───────┤ Ry(0.020584) ├┤ Sdg ├┤ H ├┤M├
«          ┌──────┴──────┐├─────────────┬┘└┬───┬┘└┬─┬┘└╥┘
«     q_1: ┤ Rz(0.70807) ├┤ Ry(0.96991) ├──┤ H ├──┤M├──╫─
«          └─────────────┘└─────────────┘  └───┘  └╥┘  ║ 
«__c_XY: 2/════════════════════════════════════════╩═══╩═
«                                                  1   0 

 with observable 
SparsePauliOp(['ZZ', 'IZ'],
              coeffs

In [5]:
from qiskit.primitives import Sampler

OpTree.evaluate.evaluate_tree_with_sampler(measured_tree,{},Sampler())

        ┌────────────────┐┌─────────────────────┐                        »
   q_0: ┤ U(0.73199,0,0) ├┤ U(0.18518,-π/2,π/2) ├─────────────────────■──»
        ├────────────────┤├─────────────────────┤┌─────────────────┐┌─┴─┐»
   q_1: ┤ U(0.59866,0,0) ├┤ U(0.04918,-π/2,π/2) ├┤ U(0,0,0.029042) ├┤ X ├»
        └────────────────┘└─────────────────────┘└─────────────────┘└───┘»
meas: 2/═════════════════════════════════════════════════════════════════»
                                                                         »
«                                  ┌────────────────────┐                  »
«   q_0: ──────────────────────■───┤ U(1.0281,-π/2,π/2) ├──────────────────»
«        ┌──────────────────┐┌─┴─┐┌┴────────────────────┤┌────────────────┐»
«   q_1: ┤ U(0,0,-0.029042) ├┤ X ├┤ U(0.18951,-π/2,π/2) ├┤ U(0,0,0.35404) ├»
«        └──────────────────┘└───┘└─────────────────────┘└────────────────┘»
«meas: 2/══════════════════════════════════════════════════════════════════»
«            

array([[-1.73167878, -1.73167878],
       [-0.53764019, -0.53764019]])

In [6]:

from qiskit.primitives import Estimator

OpTree.evaluate.evaluate_tree_with_estimator(measured_tree,{},Estimator())

array([[-1.73167878, -1.73167878],
       [-0.53764019, -0.53764019]])

In [7]:
qcnn = QCNNEncodingCircuit(4)
qcnn.convolution()
qcnn.pooling(measurement=True)
qcnn.fully_connected()
obs = CustomObservable(2, ["XY", "ZZ", "ZI", "IY"])
qcnn_obs = qcnn.QCNNObservable(obs).get_operator(np.random.rand(obs.num_parameters))
qcnn_obs

SparsePauliOp(['IXIY', 'IZIZ', 'IZII', 'IIIY'],
              coeffs=[1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j])

In [8]:
qcnn_circuit = qcnn.get_circuit(np.random.rand(qcnn.num_features),np.random.rand(qcnn.num_parameters))
print(qcnn_circuit)

     ┌─────────┐           ┌─────────┐┌─────────┐           ┌───────┐
q_0: ┤0        ├───────────┤1        ├┤0        ├───────────┤0      ├
     │  Conv_0 │┌─────────┐│         ││         │           │       │
q_1: ┤1        ├┤0        ├┤         ├┤1        ├───────────┤  FC_1 ├
     ├─────────┤│  Conv_0 ││  Conv_0 ││         │┌─────────┐│       │
q_2: ┤0        ├┤1        ├┤         ├┤  Pool_0 ├┤0        ├┤1      ├
     │  Conv_0 │└─────────┘│         ││         ││         │└───────┘
q_3: ┤1        ├───────────┤0        ├┤         ├┤1        ├─────────
     └─────────┘           └─────────┘│         ││  Pool_0 │         
c_0: ═════════════════════════════════╡0        ╞╡         ╞═════════
                                      └─────────┘│         │         
c_1: ════════════════════════════════════════════╡0        ╞═════════
                                                 └─────────┘         


c:\Users\MNH\Documents\squlearn\venv\Lib\site-packages\qiskit\circuit\quantumcircuit.py:1800: RuntimeWarning: Cregbundle set to False since an instruction needs to refer to individual classical wire
  return circuit_drawer(


In [9]:
measured_tree = OpTree.gen_expectation_tree(qcnn_circuit, qcnn_obs)

print(measured_tree)


     ┌─────────┐           ┌─────────┐┌─────────┐           ┌───────┐
q_0: ┤0        ├───────────┤1        ├┤0        ├───────────┤0      ├
     │  Conv_0 │┌─────────┐│         ││         │           │       │
q_1: ┤1        ├┤0        ├┤         ├┤1        ├───────────┤  FC_1 ├
     ├─────────┤│  Conv_0 ││  Conv_0 ││         │┌─────────┐│       │
q_2: ┤0        ├┤1        ├┤         ├┤  Pool_0 ├┤0        ├┤1      ├
     │  Conv_0 │└─────────┘│         ││         ││         │└───────┘
q_3: ┤1        ├───────────┤0        ├┤         ├┤1        ├─────────
     └─────────┘           └─────────┘│         ││  Pool_0 │         
c_0: ═════════════════════════════════╡0        ╞╡         ╞═════════
                                      └─────────┘│         │         
c_1: ════════════════════════════════════════════╡0        ╞═════════
                                                 └─────────┘         

 with observable 
SparsePauliOp(['IXIY', 'IZIZ', 'IZII', 'IIIY'],
              coeffs=[

In [10]:
from qiskit.primitives import Estimator,BackendEstimator
from squlearn import Executor

executor = Executor("qasm_simulator",shots=10000,primitive_seed=0)
estimator = executor.get_estimator()

OpTree.evaluate.evaluate_tree_with_estimator(measured_tree,{},estimator)

0.8354

In [11]:
from qiskit.primitives import BackendSampler
from qiskit_aer import Aer

#executor = Executor("qasm_simulator",shots=10000,primitive_seed=0)
#sampler = executor.get_sampler()
measured_tree = OpTree.evaluate.transform_to_zbasis(measured_tree)
OpTree.evaluate.evaluate_tree_with_sampler(measured_tree,{},BackendSampler(Aer.get_backend("qasm_simulator")))

                       ┌───┐┌────────────────┐                       ┌───┐»
   q_0: ───────────────┤ X ├┤ U(0,0,0.45607) ├──■────────────────────┤ X ├»
        ┌─────────────┐└─┬─┘├────────────────┤┌─┴─┐┌────────────────┐└─┬─┘»
   q_1: ┤ U(0,0,-π/2) ├──■──┤ U(0.78518,0,0) ├┤ X ├┤ U(0.19967,0,0) ├──■──»
        └─────────────┘┌───┐├────────────────┤└───┘└────────────────┘┌───┐»
   q_2: ───────────────┤ X ├┤ U(0,0,0.51423) ├──■────────────────────┤ X ├»
        ┌─────────────┐└─┬─┘├────────────────┤┌─┴─┐┌────────────────┐└─┬─┘»
   q_3: ┤ U(0,0,-π/2) ├──■──┤ U(0.59241,0,0) ├┤ X ├┤ U(0.04645,0,0) ├──■──»
        └─────────────┘     └────────────────┘└───┘└────────────────┘     »
meas: 4/══════════════════════════════════════════════════════════════════»
                                                                          »
«        ┌────────────┐┌─────────────┐     ┌────────────────┐                  »
«   q_0: ┤ U(0,0,π/2) ├┤ U(0,0,-π/2) ├──■──┤ U(0.96563,0,0) ├──────────────────»
« 

1.51171875